In [5]:
!pip install pyyaml

  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl (161 kB)


In [8]:
import os
import cv2
import yaml

# Base dataset path
base_path = 'D:/Group50/Facial_Emotion_Recongnition/Datasets/Preprpcess_Large_Dataset'
splits = ['train', 'val', 'test']

# Get class names from train/images subfolders
train_images_path = os.path.join(base_path, 'train', 'images')
class_names = sorted([d for d in os.listdir(train_images_path) if os.path.isdir(os.path.join(train_images_path, d))])
class_to_id = {name: idx for idx, name in enumerate(class_names)}

def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Process each split
for split in splits:
    for class_name in class_names:
        image_dir = os.path.join(base_path, split, 'images', class_name)
        label_dir = os.path.join(base_path, split, 'labels', class_name)
        ensure_dir(label_dir)

        if not os.path.exists(image_dir):
            print(f"Skipping missing folder: {image_dir}")
            continue

        for img_file in os.listdir(image_dir):
            if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue

            img_path = os.path.join(image_dir, img_file)
            image = cv2.imread(img_path)
            if image is None:
                print(f"Warning: couldn't read image {img_path}")
                continue

            h, w, _ = image.shape

            # Dummy bounding box: center of image, 50% size
            x_center = 0.5
            y_center = 0.5
            bbox_width = 0.5
            bbox_height = 0.5
            class_id = class_to_id[class_name]

            # Write YOLO label file
            label_path = os.path.join(label_dir, os.path.splitext(img_file)[0] + '.txt')
            with open(label_path, 'w') as f:
                f.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")

print("✅ Labels generated for all splits.")

# Create dataset.yaml file
yaml_data = {
    'path': os.path.abspath(base_path),
    'train': os.path.abspath(os.path.join(base_path, 'train')),
    'val': os.path.abspath(os.path.join(base_path, 'val')),
    'test': os.path.abspath(os.path.join(base_path, 'test')),
    'nc': len(class_names),
    'names': class_names
}

with open('emotion_dataset.yaml', 'w') as f:
    yaml.dump(yaml_data, f, default_flow_style=False)

print("✅ 'emotion_dataset.yaml' created.")

✅ Labels generated for all splits.
✅ 'emotion_dataset.yaml' created.


In [10]:
import os
import shutil
import random

# Base dataset path
base_dir = 'D:/Group50/Facial_Emotion_Recongnition/Datasets/Preprpcess_Large_Dataset'

# Ratios
split_ratios = {
    'train': 0.7,
    'val': 0.15,
    'test': 0.15
}

# Source folders
image_root = os.path.join(base_dir, 'train', 'images')
label_root = os.path.join(base_dir, 'train', 'labels')

# Classes (subfolders)
class_names = [d for d in os.listdir(image_root) if os.path.isdir(os.path.join(image_root, d))]

for cls in class_names:
    img_class_dir = os.path.join(image_root, cls)
    lbl_class_dir = os.path.join(label_root, cls)

    images = [f for f in os.listdir(img_class_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    random.shuffle(images)

    total = len(images)
    n_train = int(split_ratios['train'] * total)
    n_val = int(split_ratios['val'] * total)

    split_map = {
        'train': images[:n_train],
        'val': images[n_train:n_train + n_val],
        'test': images[n_train + n_val:]
    }

    for split, files in split_map.items():
        img_dest_dir = os.path.join(base_dir, split, 'images', cls)
        lbl_dest_dir = os.path.join(base_dir, split, 'labels', cls)

        os.makedirs(img_dest_dir, exist_ok=True)
        os.makedirs(lbl_dest_dir, exist_ok=True)

        moved = 0
        for file in files:
            base_name, _ = os.path.splitext(file)
            label_file = base_name + '.txt'

            img_src = os.path.join(img_class_dir, file)
            lbl_src = os.path.join(lbl_class_dir, label_file)

            img_dst = os.path.join(img_dest_dir, file)
            lbl_dst = os.path.join(lbl_dest_dir, label_file)

            if os.path.exists(img_src):
                shutil.move(img_src, img_dst)
            else:
                print(f"❌ Missing image: {img_src}")
                continue

            if os.path.exists(lbl_src):
                shutil.move(lbl_src, lbl_dst)
            else:
                print(f"⚠️ Missing label: {lbl_src}")

            moved += 1

        print(f"✅ {split.upper()} - {cls}: Moved {moved} images.")

print("\n🎉 Dataset split complete with subfolders preserved.")

✅ TRAIN - angry: Moved 3502 images.
✅ VAL - angry: Moved 750 images.
✅ TEST - angry: Moved 751 images.
✅ TRAIN - boring: Moved 1555 images.
✅ VAL - boring: Moved 333 images.
✅ TEST - boring: Moved 334 images.
✅ TRAIN - disgust: Moved 2961 images.
✅ VAL - disgust: Moved 634 images.
✅ TEST - disgust: Moved 635 images.
✅ TRAIN - fear: Moved 3545 images.
✅ VAL - fear: Moved 759 images.
✅ TEST - fear: Moved 761 images.
✅ TRAIN - happy: Moved 3554 images.
✅ VAL - happy: Moved 761 images.
✅ TEST - happy: Moved 763 images.
✅ TRAIN - neutral: Moved 3528 images.
✅ VAL - neutral: Moved 756 images.
✅ TEST - neutral: Moved 756 images.
✅ TRAIN - sad: Moved 3570 images.
✅ VAL - sad: Moved 765 images.
✅ TEST - sad: Moved 765 images.
✅ TRAIN - stress: Moved 4249 images.
✅ VAL - stress: Moved 910 images.
✅ TEST - stress: Moved 912 images.
✅ TRAIN - suprised: Moved 3570 images.
✅ VAL - suprised: Moved 765 images.
✅ TEST - suprised: Moved 765 images.

🎉 Dataset split complete with subfolders preserved.


In [14]:

from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training) 

# Train using the single most idle GPU
results = model.train(data="D:\Group50\Facial_Emotion_Recongnition\Final_Best_Codes\Training\emotion_dataset.yaml", epochs=100, imgsz=256, device=0,batch=64,pretrained=True,patience=10)

Ultralytics 8.3.153  Python-3.11.13 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\Group50\Facial_Emotion_Recongnition\Final_Best_Codes\Training\emotion_dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=

train: Scanning D:\Group50\Facial_Emotion_Recongnition\Datasets\Preprpcess_Large_Dataset\train\labels\angry... 30034 images, 0 backgrounds, 0 corrupt: 100%|██████████| 30034/30034 [00:12<00:00, 2456.24it/s]


train: New cache created: D:\Group50\Facial_Emotion_Recongnition\Datasets\Preprpcess_Large_Dataset\train\labels\angry.cache
val: Fast image access  (ping: 0.00.0 ms, read: 334.997.4 MB/s, size: 16.8 KB)


val: Scanning D:\Group50\Facial_Emotion_Recongnition\Datasets\Preprpcess_Large_Dataset\val\labels\angry... 6433 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6433/6433 [00:02<00:00, 2303.48it/s]


val: New cache created: D:\Group50\Facial_Emotion_Recongnition\Datasets\Preprpcess_Large_Dataset\val\labels\angry.cache
Plotting labels to runs\detect\train2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 256 train, 256 val
Using 8 dataloader workers
Logging results to runs\detect\train2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100      2.66G     0.3205     0.8382     0.9017         18        256: 100%|██████████| 470/470 [00:53<00:00,  8.76it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00:18<00:00,  2.81it/s]

                   all       6433       6433      0.637      0.723      0.705      0.704



100 epochs completed in 1.673 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 5.4MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 5.4MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics 8.3.153  Python-3.11.13 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)
YOLO11n summary (fused): 100 layers, 2,583,907 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00:20<00:00,  2.44it/s]


                   all       6433       6433      0.637      0.724      0.705      0.704
                 angry        750        750      0.539      0.545      0.563      0.563
                boring        333        333       0.99      0.932      0.991      0.984
               disgust        634        634      0.537      0.634      0.612      0.612
                  fear        759        759      0.615      0.596      0.643      0.642
                 happy        761        761      0.764      0.862       0.89      0.889
               neutral        756        756      0.433      0.678      0.516      0.516
                   sad        765        765       0.55      0.591       0.61       0.61
                stress        910        910      0.762      0.971      0.876      0.876
              suprised        765        765      0.539      0.702      0.643      0.643
Speed: 0.0ms preprocess, 0.3ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\detect

In [ ]:
model.export(format="onnx")

In [ ]:

from ultralytics import YOLO

# Load a model
model10 = YOLO("D:\Group50\Facial_Emotion_Recongnition\yolov10n.pt")  # load a pretrained model (recommended for training) 

# Train using the single most idle GPU
results = model10.train(data="D:\Group50\Facial_Emotion_Recongnition\Final_Best_Codes\Training\emotion_dataset.yaml", epochs=100, imgsz=256, device=0,batch=64,pretrained=True,patience=10)

Ultralytics 8.3.153  Python-3.11.13 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\Group50\Facial_Emotion_Recongnition\Final_Best_Codes\Training\emotion_dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=D:\Group50\Facial_Emotion_Recongnition\yolov10n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optim

train: Scanning D:\Group50\Facial_Emotion_Recongnition\Datasets\Preprpcess_Large_Dataset\train\labels\angry.cache... 30034 images, 0 backgrounds, 0 corrupt: 100%|██████████| 30034/30034 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.00.0 ms, read: 335.285.3 MB/s, size: 16.8 KB)


val: Scanning D:\Group50\Facial_Emotion_Recongnition\Datasets\Preprpcess_Large_Dataset\val\labels\angry.cache... 6433 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6433/6433 [00:00<?, ?it/s]


Plotting labels to runs\detect\train3\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 95 weight(decay=0.0), 108 weight(decay=0.0005), 107 bias(decay=0.0)
Image sizes 256 train, 256 val
Using 8 dataloader workers
Logging results to runs\detect\train3
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      2.34G      2.341      6.701      2.679         35        256: 100%|██████████| 470/470 [01:02<00:00,  7.48it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00:17<00:00,  2.92it/s]
